# Humidity Controll

In [2]:
%reset -f

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import psychro as psy
import pandas as pd
import va_hum

![AHU](cool_CAV.svg)

In [4]:
# constants
l = 2496e3          # specific latent heat of vaporization [J/kg]
c = 1e3             # J/(kg·K), specific heat of dry air
ρ = 1.2             # kg/m3, density of air

# parameters
ϕ_0 = 0.8           # relative humidity of outside air [-]
θ_0 = 20            # temperature of outside Air [°C]
θ_6_sp = 22         # temperature setpoint of the room [°C]
ϕ_6_sp = 0.6        # relative humidity setpoint of the room [-]

α = 0.5             # mixing ratio of outdoor air [-]
m = 5.0             # mass flow rate of supply dry air [kg/s]
β = 0.1             # mixing ratio dehumidification [-]

UA = 936 # total transmittance [W/K]
m_i = 2.120 # infiltration mass flow [kg/s]
Q_aux_s = 4000 # auxiliary heat sensible [W]
Q_aux_l = 3000 # auxiliary  latent gain [W]

Kθ, Kw = 1e15, 1e15         # controller gain

θ_s0 = 5        # °C, initial guess for saturation temperature

In [5]:
# humidity ratios
w_6_sp = psy.w(θ_6_sp, ϕ_6_sp)  # humidity ratio setpoint of the room [kg/kg]
w_0 = psy.w(θ_0, ϕ_0)  # humidity ratio of outside air [kg/kg]

In [6]:
 # Define column names (unknowns)
columns = [
    'θ1', 'w1', 'θ2', 'w2', 'θ3', 'w3', 'θ4', 'w4','θ5', 'w5','θ6', 'w6',
    'Qs_CC', 'Ql_CC', 'Qt_CC', 'Qs_HC', 'Ql_VH', 'Qs_B', 'Ql_B'
]

# Define row names (equations) with 's' for sensible and 'l' for latent
rows = [
    'MXs', 'MXl',                    # Mixing box equations
    'CCs', 'CCl', 'DH', 'DH_Q',      # Colling with dehumidification
    'MX2s', 'MX2l',                  # Mixing after dehumidification
    'HCs', 'HCl',                    # Heating coil equations
    'VHs', 'VHl',                    # Vapor humidifier equations

    'TZs', 'TZl',                   # Thermal zone equations
    'BLs', 'BLl',                   # Building equations
    'Kθ', 'Kw'                      # Controller equations
]

# Initialize coefficient matrix A and vector b
A_DH = pd.DataFrame(0.0, index=rows, columns=columns)
b_DH = pd.Series(0.0, index=rows)


In [7]:
A_DH

,θ1,w1,θ2,w2,θ3,w3,θ4,w4,θ5,w5,θ6,w6,Qs_CC,Ql_CC,Qt_CC,Qs_HC,Ql_VH,Qs_B,Ql_B
MXs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MXl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH_Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# --------------------
# Mixing box equations
# --------------------

# sensible
A_DH.loc['MXs', 'θ1'] = m * c
A_DH.loc['MXs', 'θ6'] = (1 - α) * m * c
b_DH.loc['MXs'] = α * m * c * θ_0

# latent
A_DH.loc['MXl', 'w1'] = m * l
A_DH.loc['MXl', 'w6'] = (1 - α) * m * l
b_DH.loc['MXl'] = α * m * l * w_0

# ---------------------------------------
# Cooling with dehumidification equations
# ----------------------------------------

# sensible
A_DH.loc['CCs', 'θ2'] = α * m * c
A_DH.loc['CCs', 'θ1'] = α * m * c
A_DH.loc['CCs', 'Qs_CC'] = 1
b_DH.loc['CCs'] = 0

# latent
A_DH.loc['CCl', 'w2'] = α * m * l
A_DH.loc['CCl', 'w1'] = α * m * l
A_DH.loc['CCl', 'Ql_CC'] = 1
b_DH.loc['CCl'] = 0

# dehumidification acording saturation curve
A_DH.loc['DH', 'θ2'] = psy.wsp(θ_s0)
A_DH.loc['DH', 'w2'] = -1
b_DH.loc['DH'] = psy.wsp(θ_s0) * θ_s0 - psy.w(θ_s0, 1) 

# load equation
A_DH.loc['DH_Q', 'Qt_CC'] = -1
A_DH.loc['DH_Q', 'Qs_CC'] = 1
A_DH.loc['DH_Q', 'Ql_CC'] = 1
b_DH.loc['DH_Q'] = 0

# ----------------------------------------
# Mixing after dehumidification equations
# ----------------------------------------

# sensible
A_DH.loc['MX2s', 'θ3'] = m * c
A_DH.loc['MX2s', 'θ2'] = β * m
A_DH.loc['MX2s', 'θ1'] = (1 - β) * m
b_DH.loc['MX2s'] = 0

# latent
A_DH.loc['MX2l', 'w3'] = m * l
A_DH.loc['MX2l', 'w2'] = β * m 
A_DH.loc['MX2l', 'w1'] = (1 - β) * m
b_DH.loc['MX2l'] = 0

# ----------------------
# Heating coil equations
# ----------------------

# sensible
A_DH.loc['HCs', 'θ4'] = m * c
A_DH.loc['HCs', 'θ3'] = m * c
A_DH.loc['HCs', 'Qs_HC'] = 1
b_DH.loc['HCs'] = 0

# latent
A_DH.loc['HCl', 'w4'] = m * l
A_DH.loc['HCl', 'w3'] = m * l
b_DH.loc['HCl'] = 0

# --------------------------
# Vapor humidifier equations
# --------------------------

# sensible
A_DH.loc['VHs', 'θ5'] = m * c
A_DH.loc['VHs', 'θ4'] = m * c
b_DH.loc['VHs'] = 0

# latent
A_DH.loc['VHl', 'w5'] = m * l
A_DH.loc['VHl', 'w4'] = m * l
b_DH.loc['VHl'] = 0

# ------------------------
# Thermal zone equations
# ------------------------

# sensible
A_DH.loc['TZs', 'θ6'] = m * c
A_DH.loc['TZs', 'θ5'] = m * c
A_DH.loc['TZs', 'Qs_B'] = 1
b_DH.loc['TZs'] = 0

# latent
A_DH.loc['TZl', 'w6'] = m * l
A_DH.loc['TZl', 'w5'] = m * l
A_DH.loc['TZl', 'Ql_B'] = 1
b_DH.loc['TZl'] = 0

# ------------------------
# Building equations
# ------------------------

# sensible
A_DH.loc['BLs', 'Qs_B'] = 1
A_DH.loc['BLs', 'θ6'] = (UA + m_i * c)
b_DH.loc['BLs'] = (UA * m_i * c) * θ_0 + Q_aux_s

# latent
A_DH.loc['BLl', 'Ql_B'] = 1
A_DH.loc['BLl', 'w6'] = (m_i * l)
b_DH.loc['BLl'] = (m_i * l) * w_0 + Q_aux_l

# ------------------------
# Controller equations
# ------------------------

# temperature controller
A_DH.loc['Kθ', 'θ6'] = Kθ
A_DH.loc['Kθ', 'Qs_HC'] = 1
b_DH.loc['Kθ'] = Kθ * θ_6_sp

# humidity controller
A_DH.loc['Kw', 'w6'] = Kw
A_DH.loc['Kw', 'Ql_CC'] = 1
b_DH.loc['Kw'] = Kw * w_6_sp



In [9]:
A_DH

,θ1,w1,θ2,w2,θ3,w3,θ4,w4,θ5,w5,θ6,w6,Qs_CC,Ql_CC,Qt_CC,Qs_HC,Ql_VH,Qs_B,Ql_B
MXs,5000.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500000e+03,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MXl,0.0,12480000.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,6.240000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCs,2500.0,0.0,2500.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.0,0.0,0.0,0.0,0.0,0.0,0.0
CCl,0.0,6240000.0,0.00000,6240000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0
DH,0.0,0.0,0.00038,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH_Q,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.0,1.0,-1.0,0.0,0.0,0.0,0.0
MX2s,4.5,0.0,0.50000,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2l,0.0,4.5,0.00000,0.5,0.0,12480000.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCs,0.0,0.0,0.00000,0.0,5000.0,0.0,5000.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,1.0,0.0,0.0,0.0
HCl,0.0,0.0,0.00000,0.0,0.0,12480000.0,0.0,12480000.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
b_DH

MXs     5.000000e+04
MXl     7.301279e+04
CCs     0.000000e+00
CCl     0.000000e+00
DH     -3.500718e-03
DH_Q    0.000000e+00
MX2s    0.000000e+00
MX2l    0.000000e+00
HCs     0.000000e+00
HCl     0.000000e+00
VHs     0.000000e+00
VHl     0.000000e+00
TZs     0.000000e+00
TZl     0.000000e+00
BLs     3.969040e+07
BLl     6.491484e+04
Kθ      2.200000e+16
Kw      9.895272e+12
dtype: float64

In [11]:
# Convert back to numpy arrays for solving
A_np = A_DH.to_numpy()
b_np = b_DH.to_numpy()

# Solution
x_DH = np.linalg.solve(A_np, b_np)
x_DH

LinAlgError: Last 2 dimensions of the array must be square